In [5]:
import pandas as pd
df = pd.read_csv("processed_demand.csv")
df['date'] = pd.to_datetime(df['date'])
print(df.info())
print(df.describe())
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   product_id           10000 non-null  int64         
 1   date                 10000 non-null  datetime64[ns]
 2   store_id             10000 non-null  int64         
 3   sales_quantity       10000 non-null  int64         
 4   price                10000 non-null  float64       
 5   promotions           10000 non-null  int64         
 6   seasonality_factors  10000 non-null  object        
 7   external_factors     10000 non-null  object        
 8   demand_trend         10000 non-null  object        
 9   customer_segments    10000 non-null  object        
 10  sales_7d_avg         10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(4)
memory usage: 859.5+ KB
None
         product_id      store_id  sales_quantity         price

In [6]:
# Promotions is already encoded as 0/1
# Encode categorical variables
df = pd.get_dummies(df, columns=['seasonality_factors', 'external_factors', 'demand_trend', 'customer_segments'])

# Date-based features
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week

# Drop date if not needed for training
df = df.drop(columns=['date'])


In [7]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['sales_quantity'])
y = df['sales_quantity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error 


model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)



RMSE: 8.185786723774655


In [12]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)

R² Score: 0.996628740861794


In [13]:
import joblib
joblib.dump(model, "sales_forecast_model.pkl")


['sales_forecast_model.pkl']

In [ ]:
model = joblib.load("sales_forecast_model.pkl")
predicted_sales = model.predict(new_data_df)             ## 🔁 Reusing This in LangGraph System

# ✅ Code for InventoryAgent

In [30]:
import pandas as pd
from typing import List, Dict

# Load inventory data once globally
inventory_df = pd.read_csv("processed_inventory.csv")

def InventoryAgent(forecasted_demand: List[Dict]) -> pd.DataFrame:
    """
    InventoryAgent: Evaluates reorder needs based on forecasted demand and current inventory.
    
    Args:
        forecasted_demand (List[Dict]): Each record contains:
            - product_id
            - store_id
            - date
            - forecasted_sales

    Returns:
        pd.DataFrame: With fields:
            - product_id
            - store_id
            - date
            - forecasted_sales
            - stock_levels
            - reorder_point
            - reorder_needed
            - suggested_reorder_quantity
    """

    # Convert forecast input to DataFrame
    forecast_df = pd.DataFrame(forecasted_demand)

    # Merge forecast with inventory
    merged_df = pd.merge(
        forecast_df,
        inventory_df,
        on=["product_id", "store_id"],
        how="left"
    )

    # Apply inventory evaluation logic
    def evaluate_inventory(row):
        reorder = False
        reorder_quantity = 0
        buffer_percent = 0.1  # 10% buffer

        if row['forecasted_sales'] > row['stock_levels'] or row['stock_levels'] <= row['reorder_point']:
            reorder = True
            gap = row['forecasted_sales'] - row['stock_levels']
            reorder_quantity = max(gap, 0)
            reorder_quantity += reorder_quantity * buffer_percent

        return pd.Series({
            'reorder_needed': reorder,
            'suggested_reorder_quantity': int(reorder_quantity)
        })

    # Apply function
    inventory_recommendations = merged_df.apply(evaluate_inventory, axis=1)

    # Combine final result
    result_df = pd.concat([merged_df, inventory_recommendations], axis=1)

    return result_df


## The purpose of the PricingAgent is to suggest optimized pricing for each product based on factors like:

Current stock levels

Supplier lead time

Stockout frequency

Customer segments

Forecasted demand (from CustomerAgent)

Sales trends and historical pricing



In [15]:
# pricing_agent.py

from typing import List, Dict
from datetime import datetime

def pricing_strategy(record: Dict) -> float:
    """
    Adjust price based on demand vs inventory.
    """
    price = record.get("price", 0.0)
    forecast = record.get("forecasted_sales", 0)
    stock = record.get("stock_levels", 0)
    reorder_point = record.get("reorder_point", 0)

    # Basic rule-based logic
    if forecast > stock:
        # High demand, low supply — raise price by 10%
        return round(price * 1.10, 2)
    elif forecast < reorder_point:
        # Low demand, risk of stockout — discount by 10%
        return round(price * 0.90, 2)
    elif stock > forecast * 1.5:
        # Overstocked — apply clearance discount of 15%
        return round(price * 0.85, 2)
    else:
        # Default: no change
        return price


def PricingAgent(input_data: List[Dict]) -> List[Dict]:
    """
    PricingAgent: Adjusts prices based on inventory and forecasted sales.
    
    Args:
        input_data (List[Dict]): Each record contains:
            - product_id
            - store_id
            - date
            - forecasted_sales
            - stock_levels
            - reorder_point
            - price
    
    Returns:
        List[Dict]: With fields:
            - product_id
            - store_id
            - date
            - adjusted_price
    """
    adjusted_prices = []

    for record in input_data:
        adjusted_price = pricing_strategy(record)

        adjusted_prices.append({
            "product_id": record["product_id"],
            "store_id": record["store_id"],
            "date": record["date"],
            "adjusted_price": adjusted_price
        })

    return adjusted_prices


In [16]:
if __name__ == "__main__":
    sample_input = [
        {
            "product_id": "1001",
            "store_id": 22,
            "date": "2024-07-10",
            "forecasted_sales": 350,
            "stock_levels": 300,
            "reorder_point": 200,
            "price": 75.0
        },
        {
            "product_id": "1002",
            "store_id": 10,
            "date": "2024-07-10",
            "forecasted_sales": 150,
            "stock_levels": 500,
            "reorder_point": 180,
            "price": 65.0
        }
    ]

    result = PricingAgent(sample_input)
    print(result)


[{'product_id': '1001', 'store_id': 22, 'date': '2024-07-10', 'adjusted_price': 82.5}, {'product_id': '1002', 'store_id': 10, 'date': '2024-07-10', 'adjusted_price': 58.5}]


# ✅ SupplyChainAgent – Design Overview
🎯 Goal:
Based on:

Predicted demand (from CustomerAgent)

Current stock and reorder thresholds (from InventoryAgent)

Operational parameters (from processed_pricing.csv)

It should decide:

Whether to reorder (reorder_decision)

Suggested reorder quantity (reorder_quantity)

Expected delivery time (expected_delivery_days)



In [17]:


# Load processed_pricing data
pricing_df = pd.read_csv("processed_pricing.csv")

def supply_chain_agent(input_data: Dict) -> Dict:
    # Extract fields from input
    product_id = input_data['product_id']
    store_id = input_data['store_id']
    date = input_data['date']
    predicted_sales = input_data['predicted_sales']
    stock_levels = input_data['stock_levels']
    reorder_point = input_data['reorder_point']

    # Get matching pricing info
    pricing_row = pricing_df[
        (pricing_df['product_id'] == product_id) &
        (pricing_df['store_id'] == store_id)
    ]

    if pricing_row.empty:
        raise ValueError(f"No pricing data found for Product {product_id}, Store {store_id}")

    row = pricing_row.iloc[0]

    # Extract relevant values
    supplier_lead_time = int(row['supplier_lead_time_(days)'])
    warehouse_capacity = int(row['warehouse_capacity'])
    order_fulfillment_time = int(row['order_fulfillment_time_(days)'])

    # Decision: Reorder if stock below reorder point or predicted to go below after demand
    reorder_decision = (stock_levels < reorder_point) or ((stock_levels - predicted_sales) < reorder_point)

    # Quantity to reorder: Target enough to cover demand for lead time, but not exceed warehouse capacity
    suggested_reorder_quantity = max(0, int((predicted_sales * supplier_lead_time) - stock_levels))
    reorder_quantity = min(suggested_reorder_quantity, warehouse_capacity - stock_levels)

    return {
        "product_id": product_id,
        "store_id": store_id,
        "date": date,
        "reorder_decision": reorder_decision,
        "reorder_quantity": reorder_quantity if reorder_decision else 0,
        "expected_delivery_days": supplier_lead_time + order_fulfillment_time
    }


In [51]:
print(pricing_df.head(3)  , '\n\n\n\n', inventory_df.head(3))

   product_id  store_id  price  competitor_prices  discounts  sales_volume  \
0        9502        13  31.61              56.14      19.68           255   
1        2068        77  35.51              63.04      16.88             5   
2        7103        59   6.54              30.61      10.86           184   

   customer_reviews  return_rate_(%)  storage_cost  elasticity_index  
0                 3            13.33          6.72              1.78  
1                 3             1.50          8.38              1.67  
2                 3             9.44          3.86              2.46   



    product_id  store_id  stock_levels  supplier_lead_time_(days)  \
0        9286        16           700                         10   
1        2605        60            82                         11   
2        2859        55           145                         25   

   stockout_frequency  reorder_point expiry_date  warehouse_capacity  \
0                  14            132  2024-01-15     